<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/specific-experiments/review-algoritmeregister/algoritmeregister.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#if running in colab and you want to reproduce, run the following two, otherwise ignore this
!pip3 install wetsuite -U

In [ ]:
!python3 -m spacy download nl_core_news_lg

## Purpose of this notebook

For context, [algoritmes.overheid.nl](https://algoritmes.overheid.nl) is government organisations 
making a list about the algorithms that assist their day-to-day,
from automation to machine learning, in a bid to be more transparent towards citizens.

What if, based on just the descriptions,
we want to see whether each might end up profiling individuals? 

We expect this primarily in applications related to social security, yet there may be more.
Can we use our own algorithm to tell, or at least make a pre-selection to look at?

### Considering that task

Yes, [yhe algoritmes.overheid.nl site](https://algoritmes.overheid.nl) has topic filtering, 
and actually a decent search, but it's hard to know how complete the results are at any time.

And yes, it has have text per case, in sections based on a [publication standard asking some specific questions](https://algoritmes.pleio.nl/wiki/view/35c7eeaa-deee-47d3-b2cc-88f7c90b475c/invullijst-algoritmeregister-publicatiestandaard-100), it's in a few different places, 
and the thoroughess (and e.g. evasiveness) of the answer does vary.

Even though there are currently 'only' five hundred items to check,
reviewing by hand would be slow busywork - we would be two thousand clicks further before we reviewed everything.
And then maye you would want to do the same next week. Let's try to make our own life simpler.

We described three things to address:
- turning five hundred _things_ on a site into something we can consume more easily 
- ensure that we are pick up all cases we care about
- classify as worrisome or not

Consuming the site as data is not too hard - the site itself has [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) and [XLSX](https://en.wikipedia.org/wiki/XLSX) export,
linked on each result page. 
Both of these are machine-readable enough, so we can start handling that as text data.
Sadly you cannot export the current filtered selection, so we will have to deal with all of it.

In theory you can see the other two point as classification tasks.
- social topic or not? (as a filter)
- has worrisome phrasing or not? (probably to score)

While classification is automatic, in this case there is little text to work on.

This matters in that longer documents make it more likely that _something_ gives indirect hints; with just a few sentences that may simply not be thre.


This also makes phrasing important - just hedging your language well enough would probaly make something pass.
We suspect that even _if_ there were no issue in the time cost in reading all that,
people doing that busywork would probably still have trouble, and might even disagree.

Sure, clearer cases may have some red-flag words --  _but_ some cases are just too vague even forwell-informed human estimation. 
When the information isn't there for us even with a little more real-world knowledge, it's probably less there for machines.

In particular the last case of the next list suggests a real difference between 
"lack of topical words that make us worry; probably fine", and "lack of topical words; should ask for them".

---

Consider some real cases that **we would consider investigating further**:

* [Model van Bijstand naar Werk](https://algoritmes.overheid.nl/nl/algoritme/model-van-bijstand-naar-werk-gemeente-den-haag/71627856#verantwoordGebruik)
  - worrisome: 
    - the topic of 'bijstand'
    - terms like 'op maat', 'dossiers', 'decision tree', 'bepaalde segmenten'
    - the goal is to invite _specific_ people
  - alleviates:
    - 'eenmalig' suggests removal after suggestion
    - system _advises_ an invitation, does not decide 

* [Heronderzoeken Uitkeringsgerechtigden](https://algoritmes.overheid.nl/nl/algoritme/heronderzoeken-uitkeringsgerechtigden-gemeente-rotterdam/36585638#verantwoordGebruik)
  - worrisome: 
    - the topic of 'uitkering'
    - terms like 'kansberekening', 'risico-inschattingsgetal', 'historische gegevens', 'voorspellend', 'gezinssituatie'
  - alleviates: 
    - system _advises_ a review, does not decide or execte one?

* [Onderzoekswaardigheid: Slimme check levensonderhoud](https://algoritmes.overheid.nl/nl/algoritme/onderzoekswaardigheid-slimme-check-levensonderhoud-gemeente-amsterdam/95794697#verantwoordGebruik)
  - worrisome: 
    - terms like 'score', 'onderzoek', 'leeftijd', 'geboorteland'
  - alleviates:
    - system _advises_ a review, does not make decisions  ('onderzoekswaardig')
    - (explains that it addresses ethics in that it should be more equal-opportunity about scrutiny, though not exactly how much profiling is or isn't involved in that)

* [Werkverkenner](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#verantwoordGebruik)
  - worrisome:
    - topic of 'uitkering'
    - terms like 'score', the [15-item list of personal information that it uses](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#werking) 
  - alleviates:
    - said to involve checking the information that led to the invitation
    - external verification of ethics

---

**Possibly okay?**

* [Wmo-voorspelmodel](https://algoritmes.overheid.nl/nl/algoritme/wmovoorspelmodel-gemeente-den-haag/97246956#verantwoordGebruik)
  - worrisome: 
    - terms like 'voorspelmodel' 
  - alleviates: 
    - results are about collective, not personal use
    - basis might still be personal info but 'open data' suggests not

* [Vroegsignalering](https://algoritmes.overheid.nl/nl/algoritme/vroegsignalering-gemeente-roosendaal/73933449#verantwoordGebruik)
  - worrisome: 
    - terms like 'schuldeisers', 'automatisch', 'persoonsgegevens'
  - alleviates: 
    - 'advies' suggests suggestion rather that decision system
    - 'handmatig' suggests this is about automating, not about doing things particularly differently

* [Rechten Rotterdammers](https://algoritmes.overheid.nl/nl/algoritme/rechten-rotterdammers-gemeente-rotterdam/33569518#verantwoordGebruik)
  - worrisome: 
    - topic like uitkering, 'beslisregels', 'toekennen' based on its output
  - alleviates: 
    - terms like 'advies', 'medewerker'+'ondersteunt'
    - 'Regelgebaseerd algoritme stelt rechten vast (geen risicoscores) '

* [Automatische kwijtschelding](https://algoritmes.overheid.nl/nl/algoritme/automatische-kwijtschelding-gemeente-rotterdam/54699221#verantwoordGebruik)
  - worrisome: 
    - terms like 'combineren'+'informatie', 'inkomstengegevens', 'financiële' en 'huishouden' 
  - alleviates: 
    - terms like 'opt-in'; 'handmatig'

* [IPA risicomodel](https://algoritmes.overheid.nl/nl/algoritme/ipa-risicomodel-nederlandse-arbeidsinspectie/74441495#verantwoordGebruik)
  - worrisome: 
    - terms like 'risicomodel', 'meldingen', 'inspectie'
  - alleviates: 
    - the fact that this is about asbestos, not people

---

**Unsure?**

* [DIAfragma](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#verantwoordGebruik)
  - worrisome:
    - terms like 'integraal klantbeeld', 'BSN-nummers', 'geslacht', 'koppeling',  [list of 10+ pieces of personal information](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#werking) 
  - alleviates: 
    - specifically avoids using some information that might lead to discrimination?

* [Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning)](https://algoritmes.overheid.nl/nl/algoritme/sociaal-domein-pko-kennissystemen-proces-kennisondersteuning-gemeente-arnhem/65732191#verantwoordGebruik)
  - worrisome: 
    - topic of uitkering
    - terms like 'beslisboom', 'controleert', 'door het systeem getrokken conclusies' 
  - alleviates: 
    - 'nalopen'
    - not a lot of contentful words to go on, though?




---

What if we limit ourself to marking how interesting cases are for us to potentially _look at_,
rather that decide _how bad they are_?

We only support a human in making their decisions easier
(the same thing we want to see in some of these algoritms themselves),

To scrape as much as we can from minimal text, 
we might look at worrisome words, which verbs appear in the same sentence? Paragraph?

There sould probably be some suggestion of phrases to include,
based on being related to what you have already decided is good or bad,
or just on being contentful words not currently under consideration.

---
That said, this approach also comes with different fundamental limitations. 

For example, even smarter NLP might miss negations (even recent developments, LLMs, used to be infamously bad at seeing negations).

Consider detecting 'geautomatiseerde besluitvorming' and miss that the words before were 'Er is geen sprake van';
- You might see 'BSN-nummers' and 'koppeling' but miss the words 'zonder verdere' inbetween.
- You might see 'handmatig' but not tell whether it is the thing we are not doing anymore, or ensuring is still there.

Things like "Risico's bij het gebruik van het algoritme zijn mede vanwege de mogelijkheid om in te grijpen op de door het systeem getrokken conclusies niet noemenswaardig, waarmee de proportionaliteit in orde is." are a feat of sentence nesting, and we would like an extra connecting argument that actually leads to that final "it's probably fine".


But then, if we're not even looking for interpretation like negatives, we are _certain_ to miss those differences.
That said, it is not necessarily a bad thing to bring up these cases, if only because some cases lie fully in context.

"Het algoritme kan alleen automatisch een aanvraag/aangifte goedkeuren. Een aanvraag/aangifte afkeuren kan alleen de ambtenaar." may sound like good bias, or bad,
depending on what it is we're okaying.

## The data

### Fetching and parsing data

In [1]:
import csv, io, random, re, math
import wetsuite.helpers.net
import wetsuite.helpers.strings
import wetsuite.helpers.spacy
import wetsuite.helpers.lazy

In [2]:
## Fetch that data as CSV
csv_bytes = wetsuite.helpers.net.download( 'https://algoritmes.overheid.nl/api/downloads/NLD?filetype=csv', timeout=60 )   # higher timeout because it may take take 10 to 20 seconds to produce that
# (you can ignore this) 
# (do some manual decoding because it seems to has UTF-8 with a BOM which is not invalid but is sort of pointless, and not something a lot of things look for / look to remove)
csv_str = csv_bytes.decode('utf-8').lstrip('\ufeff') # Remove that, move on.


## Parse that CSV
# Now we have CSV in a (unicode) string.
# CSV is not a single standard, it has flavours that we need to get the right one of.
# This seems good for this data:
class AlgoritmRegisterCSVDialect(csv.Dialect):
    header         = True
    lineterminator = '\n'
    delimiter      = ','
    doublequote    = True
    quotechar      = '"'
    quoting        = csv.QUOTE_ALL

csv_parsed = csv.DictReader( io.StringIO(csv_str) , dialect=AlgoritmRegisterCSVDialect())
algodict_list = list( csv_parsed ) 

In [3]:
# we now have a list of dicts. Just to point that out, show the first:   (note that all items will share the same keys, due to where it came from)
algodict_list[0] 

{'name': 'Public Eye',
 'organization': 'Gemeente Amsterdam',
 'description_short': 'Amsterdam is een drukke stad. Dit kan soms leiden tot verkeersonveilige situaties. Door data te verzamelen over de aantallen voetgangers is het mogelijk om maatregelen te treffen, waardoor de drukte in goede banen geleid kan worden.',
 'category': 'Organisatie en bedrijfsvoering, Economie, Ruimte en infrastructuur',
 'website': 'https://algoritmeregister.amsterdam.nl/public-eye/',
 'status': 'Buiten gebruik',
 'goal': 'Amsterdam is een drukke stad. Dit kan soms leiden tot verkeersonveilige situaties. Door data te verzamelen over de aantallen voetgangers is het mogelijk om maatregelen te treffen, waardoor de drukte in goede banen geleid kan worden. Hierdoor blijft de stad comfortabel, bereikbaar en verkeersveilig. Als een situatie door te grote drukte onveilig wordt, kan de gemeente ingrijpen. Dit gebeurt bijvoorbeeld door digitale informatieborden te plaatsen, zodat mensen weten welke routes ze moeten 

### Inspecting data

In [3]:
print( 'Number of algorithm:', len(algodict_list) )

Number of algorithm: 771


In [ ]:
# ...so showing _all_ data like above would be messy. Let's try something more compact, like a table.
# pandas is a handy tool to work with table-like data
#   and deals with lists of dicts (as we just read in) by making a column for each key -- so in this case happens to need no further instruction

import pandas 
# just for this case, show just a few rows, 
#   ...but all columns - we're pointing out how sparse this is, and a lot of it not directly useful
with pandas.option_context('display.min_rows',5,  'display.max_columns',None): 
    df = pandas.DataFrame(algodict_list) 
    display(df)

,name,organization,description_short,category,website,status,goal,proportionality,lawful_basis,standard_version,url,contact_email,lang,publiccode,source_data,methods_and_models,human_intervention,risks,provider,process_index_url,tags,source_id,begin_date,end_date,impacttoetsen,publication_category,lawful_basis_grouping,impacttoetsen_grouping,source_data_grouping,algorithm_id
0,Algoritme subsidieregeling Tegemoetkoming Ener...,Rijksdienst voor Ondernemend Nederland,Het algoritme wordt gebruikt om voor alle aanv...,Economie,,Buiten gebruik,\r\nMet de inzet van het algoritme wordt het n...,\r\nDe inzet van het algoritme bij TEK-aanvrag...,\r\nHet algoritme wordt ingezet voor taken uit...,1.0,,Via contactformulier: https://www.rvo.nl/onder...,nld,,\r\nIn het algoritme worden gegevens over omze...,\r\n\r\nRVO zet een algoritme in als ondersteu...,Het risicomodel bepaalt van elke aanvraag he...,\r\nHet risicomodel wordt ter goedkeuring voor...,,,"Tegemoetkoming Energiekosten, TEK, Subsidiereg...",,2023-04,2025-01,DPIA is uitgevoerd,Hoog-risico AI-systeem,"1: Wettelijke grondslag, https://wetten.overhe...",1: Data Protection Impact Assessment (DPIA),,42464349
1,TVL Risicomodel,Rijksdienst voor Ondernemend Nederland,Het algoritme wordt gebruikt om voor alle aanv...,"Economie, Sociale zekerheid",,Buiten gebruik,Met de inzet van het algoritme wordt het nemen...,In de TVL wordt het risicomodel (het algoritme...,Wettelijke grondslag TVL regeling: https://wet...,1.0,,Voor contact over deze registratie neem contac...,nld,,,,Bij elke nieuwe TVL ronde (aanvraagfase zowel ...,,,,,,2020-12,2025-01,,Hoog-risico AI-systeem,,,,62129549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,iBurgerzaken e-diensten,Gemeente Urk,Inwoners en ondernemers van de gemeente Urk ku...,Organisatie en bedrijfsvoering,https://www.urk.nl\t,In gebruik,Het doel van de e-diensten met het onderliggen...,Alle aanvragen via de balie afhandelen is voor...,,1.0,,info@urk.nl,nld,,"Gegevens die worden gebruikt komen uit de BRP,...",Via de website van de gemeente kan een inwoner...,Het algoritme gaat uit van een positief scenar...,Het is aan gemeenten om vorm te geven aan het ...,PinkRoccade LG,,,,2014-06,n.v.t.\t,,Overige algoritmes,"1: Wet basisregistratie personen, https://wett...",,"1: BRP, Rijksdienst voor Identiteitsgegevens. ...",81854891
770,Eerste Hulp bij Geldzaken,Gemeente Groningen,Eerste Hulp bij Geldzaken is het zo vroeg moge...,Sociale zekerheid,https://gemeente.groningen.nl/eerste-hulp-bij-...,In gebruik,Het doel van het algoritme is vroegsignalering...,De gemeente krijgt allerlei signalen van ander...,De gemeente ontvangt de signalen en biedt onde...,1.0,https://gemeente.groningen.nl/open-data,opendata@groningen.nl,nld,,De gemeente krijgt zes soorten signalen door:1...,De gemeente Groningen gebruikt de webapplicat...,Na het ontvangen van het signaal volgt er een ...,De gemeente verwerkt alle gegevens volgens de ...,xxllnc,,"Vroegsignalering, schuldhulpverlening, schuld,...",,04-2020,,,Impactvolle algoritmes,"1: Wet gemeentelijke schuldhulpverlening, http...",1: DPIA,1: Meer uitleg over de gebruikte gegevens en v...,55597555


In [5]:
# inspect how common some values are in each field
display( df.value_counts('status') )

status
In gebruik         683
In ontwikkeling     49
Buiten gebruik      39
Name: count, dtype: int64

In [6]:
display( df.value_counts('publication_category') )

publication_category
Overige algoritmes        375
Impactvolle algoritmes    369
Hoog-risico AI-systeem     27
Name: count, dtype: int64

In [7]:
catcounts = df.value_counts('category')
display( catcounts[catcounts>1] )

category
Organisatie en bedrijfsvoering                                             288
Sociale zekerheid                                                           76
Openbare orde en veiligheid                                                 67
                                                                            57
Overheidsfinanciën                                                          46
Ruimte en infrastructuur                                                    32
Verkeer                                                                     32
Natuur en milieu                                                            18
Zorg en gezondheid                                                          14
Wonen                                                                       11
Economie, Ruimte en infrastructuur                                           8
Economie                                                                     7
Recht                                      

## Working our way to do that task

### Selecting just the interesting text fields

In [8]:
# This is the list we have
print( 'Columns/key names:', list( algodict_list[0].keys() ) )

Columns/key names: ['name', 'organization', 'description_short', 'category', 'website', 'status', 'goal', 'proportionality', 'lawful_basis', 'standard_version', 'url', 'contact_email', 'lang', 'publiccode', 'source_data', 'methods_and_models', 'human_intervention', 'risks', 'provider', 'process_index_url', 'tags', 'source_id', 'begin_date', 'end_date', 'impacttoetsen', 'publication_category', 'lawful_basis_grouping', 'impacttoetsen_grouping', 'source_data_grouping', 'algorithm_id']


In [9]:
# Looking at that short table above, most keys do note seem interesting.
#  starting from the list of keys above, remove things we do not find interesting.
#  the commented-out things are still _potentially_ useful - play with it to see
our_key_choices = [
        'category',
        'algorithm_id',           # keep this around, it lets us later link to the site again
        #'publication_category',
        'name', 
        'organization',
        'description_short',
        'goal',
        'proportionality',
        'source_data',
        'methods_and_models',
        'human_intervention',
        'risks',
        'tags',
        #'status',
        #'begin_date', 'end_date',
        #'impacttoetsen',
        #'impacttoetsen_grouping',
        #'source_data_grouping', 
]

# do the selection as a pandas table, so we can get a quick overview of what we made
smaller_table = df[our_key_choices]
display(smaller_table)

,category,algorithm_id,name,organization,description_short,goal,proportionality,source_data,methods_and_models,human_intervention,risks,tags
0,Economie,42464349,Algoritme subsidieregeling Tegemoetkoming Ener...,Rijksdienst voor Ondernemend Nederland,Het algoritme wordt gebruikt om voor alle aanv...,\r\nMet de inzet van het algoritme wordt het n...,\r\nDe inzet van het algoritme bij TEK-aanvrag...,\r\nIn het algoritme worden gegevens over omze...,\r\n\r\nRVO zet een algoritme in als ondersteu...,Het risicomodel bepaalt van elke aanvraag he...,\r\nHet risicomodel wordt ter goedkeuring voor...,"Tegemoetkoming Energiekosten, TEK, Subsidiereg..."
1,"Economie, Sociale zekerheid",62129549,TVL Risicomodel,Rijksdienst voor Ondernemend Nederland,Het algoritme wordt gebruikt om voor alle aanv...,Met de inzet van het algoritme wordt het nemen...,In de TVL wordt het risicomodel (het algoritme...,,,Bij elke nieuwe TVL ronde (aanvraagfase zowel ...,,
2,Natuur en milieu,44919492,Subsidieregeling Brede weersverzekering (BWV),Rijksdienst voor Ondernemend Nederland,Het algoritme wordt gebruikt voor het verstrek...,Het doel van dit algoritme is de afhandeling v...,Automatische berekeningen zijn sneller dan met...,De aanvrager levert de volgende gegevens aan d...,Het algoritme bepaalt of er subsidie voor een ...,Er is geen menselijke tussenkomst. De aanvrage...,Het ontwerp van algoritmes controleren we in e...,"RVO, Brede weersverzekering, BWV"
3,"Organisatie en bedrijfsvoering, Recht",21544327,Anonimiseringstool,Gemeente Hilversum,Het algoritme in de software is voornamelijk i...,De anonimiseringstool wordt ingezet om invulli...,Het komt voor dat tekstfragmenten in documente...,In het begin van het gebruik werd een inrichti...,Op basis van slimme regels doorzoekt de softwa...,De software werkt op basis van een inrichtings...,Om het risico dat documenten onvoldoende worde...,"Anonimiseren, informatieverzoek"
4,"Sociale zekerheid, Werk",15759870,Risicoscan Verwijtbare Werkloosheid,Uitvoeringsinstituut Werknemersverzekeringen,Helpt ons om mogelijke verwijtbare werklooshei...,"Wanneer u een WW-uitkering bij ons aanvraagt, ...",De risicoscan is ontwikkeld om verwijtbare wer...,De risicoscan gebruikt gegevens die nodig zijn...,\nDe risicoscan stelt op basis van een combina...,\nOnze medewerkers houden op de volgende manie...,\nWe zorgen ervoor dat we blijven voldoen aan ...,
...,...,...,...,...,...,...,...,...,...,...,...,...
766,Natuur en milieu,39981992,Vegetatie algoritme (Iteratio 2),Provincie Zuid-Holland,ITERATIO is een applicatie voor het maken van ...,Vegetaties op het land en in het water zijn de...,Het is een state-of-the art model om vanuit ve...,"vegetatiekaart, vegetatietypologie, vegetatieo...",ITERATIO is een computerprogramma dat berekeni...,Er is geen direct beleidsmatig gevolg van het ...,"Het algoritme kan mogelijk fouten bevatten, wa...","natuurbeheer, vegetatie"
767,Migratie en integratie,64727572,Zoeken naar mogelijke registraties van vreemde...,Ministerie van Justitie en Veiligheid,Algoritme dat het mogelijk maakt om fonetisch ...,\nHet bieden van hulp bij het onderzoeken of v...,Het algoritme kan worden gebruikt om gegevens ...,In de database zijn uitsluitend persoonsgegeve...,Bij registratie of wijziging van gegevens van ...,Het algoritme wordt alleen gebruikt ter onders...,Het algoritme wordt jaarlijks getest om te zie...,
768,"Wonen, Openbare orde en veiligheid, Ruimte en ...",93577449,Versterking scenario-afweging voor woningen di...,Nationaal Coördinator Groningen,Indien woningen niet op norm zijn verklaard wo...,Het doel van de berekening is om een scenario ...,De voordelen bestaan uit een zo eerlijk en gel...,Investeringskosten bouwkundig versterken (BKV)...,De scenario afweging heeft als doel om een sce...,Alle scenario bepalingen betreffen een advies ...,"Het risico wordt verkleind, door menselijke tu...","Groningen, Aardgaswinning, Versterken, NCG, Na..."
769,Organisatie en bedrijfsvoering,81854891,iBurgerzaken e-diens

### Making a new structure with that data, and some new fields

In [10]:
# Say we want to flatten/merge all that text to one string
# The column selection part is easy to do in pandas (we just did, above),
#   but addressing rows in pandas is awkward,
#   so we go back to the earlier data in python types, lis.  Same key names, after all.

ourdata = {}  # { algo_id : {  key:value, key:value, ... } }
for item_dict in algodict_list:
    algo_id = item_dict.get('algorithm_id')

    # We make one new field, which contains all interesting text, to work on later. This code pastes them together.
    text_fragments = [] 
    for text_field_name in our_key_choices: 
        if text_field_name not in ('algorithm_id'):
            text_fragments.append( item_dict.get(text_field_name) )

    ourdata[algo_id] = { # we may not use all these fields, but it's handy
        'algo_id':algo_id, 
        'titel':item_dict.get('name'),
        'category':item_dict.get('category'),
        'textlist':text_fragments, 
        # preparing for what we're doing later:
        'sociaal':set(), 
        'worrying':set(),
        'alleviates':set(),
    }

In [14]:
# show a random one of them, to show what we just made
display( random.choice( list(ourdata.items()) ) )

('91386595',
 {'algo_id': '91386595',
  'titel': 'Anonimiseringssoftware',
  'category': 'Organisatie en bedrijfsvoering',
  'textlist': ['Organisatie en bedrijfsvoering',
   'Anonimiseringssoftware',
   'Gemeente Noordwijk',
   "Het algoritme onderstreept de persoonsgegevens in documenten. Een medewerker moet alle pagina's bekijken en controleren of het document goed geanonimiseerd is. Daarna verwijdert de software alle gemarkeerde informatie en wordt het zwartgelakt. Daarna kunnen de documenten gepubliceerd worden, bijvoorbeeld op basis van de Wet Open Overheid (WOO).",
   'De anonimiseringssoftware wordt ingezet om documenten die de gemeente publiceert sneller en beter te anonimiseren. Zo voorkomen we datalekken en dragen we bij aan een betere bescherming van de AVG-rechten van betrokkenen.',
   'De gemeente Noordwijk moet steeds vaker informatie openbaar maken. Daarom moet privacy- of bedrijfsgevoelige informatie worden weggelakt. Het voordeel van de anonimiseringssoftware is dat e

### Aside (you can skip this): Noting the 'enthusiastic dutch compounding makes for out-of-vocabulary words' problem

NLP likes to see words as 'character sequences without spaces' because that's a lot easier to tokenize into (and then assign a unique number for each token).

Dutch likes smashing together words any chance it gets, which means it's better than other languages
at making words that aren't very common in training data, known at all, and/or are hard to estimate meaning of.
And there are ways around this but they have to be done intentionally.

We normally might not care so much, because for larger documents we are likely to catch the topic 
even if we miss a few words, but here we might need every word we can get. 

A thing to keep in mind. We may use libaries like spacy sparingly/informedly. 
<!--
#     # for nc in doc.noun_chunks:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( nc.text ) )
#     # for ent in doc.ents:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( ent.text ) )
#     # if len(ph)>0:
#     #     data[id]['ph'].extend( ph )

#     # data[id]['ph'] = ( wetsuite.helpers.strings.ordered_unique(data[id]['ph'], case_sensitive=False) )

# pprint.pprint( data )
# -->

In [50]:
# get a random example case
random_case_dict = random.choice( list(ourdata.values()) )
random_case_text = ' '.join( random_case_dict['textlist'] )

doc = wetsuite.helpers.lazy.spacy_parse( random_case_text )
display( wetsuite.helpers.spacy.notebook_content_visualisation(doc) )  # that function already happens to highlight out-of-vocabulary words

### Filtering text for sociale zekerheid; identifying words of interest

Assuming we don't trust the field/filter the site gives us, 
or at least want to check it leaves out nothing, let's try our own, and compare.

Right now these are used as substrings, so will match even if part of larger words (which is why inflections are removed from this list, or should be),
done intentionally to alleviate the compounding issue - e.g. bijstand will match bijstandnorm, bijstandgerechtigd, etc.  
There's some extra code to then extract whatever the whole word was we matched a part from.

In [15]:
# substrings that indicate this is related to social security
social_indicators = '''bijstand
uitkering
werkloosheidswet
IOAZ
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen 
IOAW
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers
IOW
Wet inkomensvoorziening oudere werklozen
de Wmo
Wet maatschappelijke ondersteuning
WWB
wet werk en bijstand
Wet LB
Wet op de loonbelasting
Awir
Algemene wet inkomensafhankelijke regelingen
Ziektewet
Participatiewet
UWV
AOW
arbeidsongeschik
arbeidsbeperk
arbeidsvermogen
WAJONG
schuldhulpverlen
loonwaarde
levensonderhoud
re-integratie
reïntegratie
werkhervat
Hulp bij het Huishouden
Ondersteuning thuis
betalingsachterstand
Kinderbijslag
kinderopvang
toeslagpartner
Werk en Inkomen
ziekmeld
Rechtsbijst
kwijtschel
zorgaanbied
ziekengeld
woonlandbeginsel
alimentatie
Sociale Verzekerings
Sociale zekerheid'''.lower().split('\n') # ww   sociale


# substrings that make us pay attention.  Again: this is NOT very robust
worrisome_indicators = '''voorspel
beslis
burgerservicenummer
BSN
Sofinummer
Wet basisregistratie personen
BRP
profiler
individu
risico
score
gestuurd
onderzoek
kansberekening
inschattingsgetal
dossier
uw gegevens
combineren
woonadres
inkomst(en)?gegeven
vermogensgegeven
voertuiggegeven
financi[eë]le gegeven
financi[ëe]le informatie
gezinss?ituatie
leeftijd
functie
huishouden
geboorte
persoonsgegeven
naamgegeven
geboortegegeven 
ziekmelding
leeftijd
geboorteland
nationaliteit
decision tree
beslisbo
rule-based
op maat
bijstand
uitkeringsgerechtig
werkzoeken
zonder tussenkomst van
historische gegeven
schuldeiser
schuldenaa?r
toeslagen
heronderzoek
datamodel'''.lower().split('\n') # heronderzoek


# substrings that suggest people considered (not) profiling
alleviating_indicators = '''handmatig
handmatig bekijk
handmatige? controle
niet leid
advie?[sz]
eenmalig
ondersteun
foutgevoel
Algemene Verordening Gegevensbescherm
AVG
profilering
discrimin
kwijtscheld
niet zelfler
privacy
privacy( impact)? assessment'''.lower().split('\n')

### Match those terms in that data

In [16]:
# we could work on only the things already tagged 'sociale zekerheid' to make the output shorter
#filtered_data = {algoid:details  for algoid, details in ourdata.items()  if 'sociale zekerheid' in details.get('category').lower()}
# ...but let's just work on all instead  (right now we still filter later, based on whether we found any interesting words)
filtered_data = ourdata

In [ ]:
def find_whole(pattern, text): # helper function
    ''' let us match by substring that ends in a word, but return the rest of the word as well,   
        e.g. when looking in term 'sociale zekerheid en zo' with pattern 'soci', we get back 'sociale':
        e.g. add_whole( "soci", "sociale zekerheid" ) == "sociale" 
    '''
    match = re.search(r'\b'+pattern+r'[\w]*', text)
    if match is not None:
        return [match.group(0)]
    else:
        return []

for algo_id, details in filtered_data.items():
    category = details.get('category')
    textlist = details.get('textlist')
    text = '  '.join(textlist).lower()  

    # find interesting words, add to the case's fields
    for szi in social_indicators:
        details['sociaal'].update(    find_whole( szi, text )  )
    for wi  in worrisome_indicators:
        details['worrying'].update(   find_whole( wi,  text )  )
    for ai  in alleviating_indicators:
        details['alleviates'].update( find_whole( ai,  text )  )

    # WARNING: DUMB SCORE -- to calculate anything even resembling a real store would take a lot more than this
    score = round(   math.log( 1+ len(details['sociaal']))  +  math.log(1+len(details['worrying']))  -  len(details['alleviates'] ),   1)
    details['dumb_review_score'] = round(score,1)

## Show in a nice table

We use pandas again, mostly because it lets us control formatting

In [19]:
with pandas.option_context('display.min_rows', 1000, 'display.max_rows', 1000): # don't truncate on viewing this table
    df = pandas.DataFrame(filtered_data.values()) # make it a pandas table again

    display( 
        df
        # filter out 'textlist' and also forces column ordering  order
        [['category', 'algo_id', 'titel', 'sociaal', 'worrying', 'alleviates', 'dumb_review_score']] 

        # select only items that had _any_ words we considered it indicating a social security thing:
        #  NOTE: comment out this line to see ALL algorithms
        [ (df['sociaal'].str.len()>0) ] 

        # sort by category, then score (to see the 'Sociale zekerheid' cases grouped / see most troublesome per group)
        #  NOTE: so look within each category, not just the top of the table!
        #        ...or remove 'category' from that list, if you don't care about the per-category part
        .sort_values(['category','dumb_review_score'],ascending=False) 
        
        # display the table slightly prettier:
        .style.format({ 
            'algo_id':    lambda x:'<a href="https://algoritmes.overheid.nl/nl/algoritme/IGNOREME/%s#algemeneInformatie">%s</a>'%(x,x), # make algorithm IDs link to the website
            # (the rest of these are minor formatting, 
            'sociaal':    lambda x:', '.join(x),    # format ['a', 'list'] like   a,list
            'worrying':   lambda x:', '.join(x),
            'alleviates': lambda x:', '.join(x),
            'dumb_review_score': lambda x:'%.1f'%x   # just one digit is enough, actually
        })
        .hide(axis="index") # hide pandas's index column
    )

category,algo_id,titel,sociaal,worrying,alleviates,dumb_review_score
Zorg en gezondheid,13669212,Wmo voorspelmodel,"ondersteuning thuis, de wmo, hulp bij het huishouden","individuele, huishouden, risico, combineren, voorspelmodel, leeftijdscategorieën","privacy, ondersteuning",1.3
Zorg en gezondheid,31934363,Signalering Misbruik en Oneigenlijk gebruik Wmo & Jeugdwet,"zorgaanbieder, de wmo","beslissing, onderzoek, risico, profilering, brp, bsn","handmatige, profilering",1.0
Werk,74441495,IPA risicomodel,uwv,"risicomodel, score, bsn, voorspellen",,2.3
Werk,42117256,Risicomodel omzetdaling NOW,uwv,"voorspellingen, functies, dossiers, beslissing, onderzoek, risicomodel, individueel","adviseert, ondersteunen",0.8
"Sociale zekerheid, Werk, Overheidsfinanciën",86726997,Toekenning Bijverdienbeloning,"uitkering, bijstandsuitkering, participatiewet, sociale zekerheid, ioaz, ioaw","dossiersniveau, financiele gegevens, toeslagen, bijstandsuitkering, risico",,3.7
"Sociale zekerheid, Werk",15759870,Risicoscan Verwijtbare Werkloosheid,"uwv, sociale zekerheid, uitkering","onderzoek, risicoscan, leeftijd, dossier",privacy,2.0
"Sociale zekerheid, Werk",11248112,Werkverkenner,"uwv, sociale zekerheid, uitkering, arbeidsongeschikt","werkzoekenden, beslist, burgerservicenummer, functie, uw gegevens, score, bsn, leeftijd","privacy, ondersteunen, advies",0.8
"Sociale zekerheid, Werk",79826945,Sollicitatiescan WW,"uwv, sociale zekerheid, uitkering","onderzoeken, persoonsgegevens, beslist, burgerservicenummer, uw gegevens, score, bsn, leeftijd","adviesbureau, privacy, ondersteuning",0.6
"Sociale zekerheid, Organisatie en bedrijfsvoering",65732191,Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning),"bijstand, werk en inkomen, uitkeringsaanvragen, sociale zekerheid","bijstand, individuele, risico, brp, beslissen, beslisboom, heronderzoeken",ondersteunt,2.7
Sociale zekerheid,16931972,Sociaal Domein: eDiensten voor aanvragen,"bijstand, uitkeringen, werk en inkomen, sociale zekerheid","bijstand, individuele, risico, burgerservicenummer, brp, beslisbomen, bsn, voorspelt",,3.8
